<a href="https://colab.research.google.com/github/AkshataKurane/Shark-Tank/blob/main/Streamlit_Trancription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Streamlit and ngrok
!pip install streamlit
!pip install pyngrok
!pip install azure-cognitiveservices-speech
!pip install reportlab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.0 MB/s eta 0:00:00


In [ ]:
import os
import time
import streamlit as st
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import azure.cognitiveservices.speech as speechsdk
from pyngrok import ngrok
import tempfile
import io


In [47]:
from pyngrok import ngrok, conf
# Replace with your actual ngrok authtoken
NGROK_AUTHTOKEN = "2nVqzn0Zd0IRxRssPVWIeWd9I7m_3JVCAYER33hYbbXni2fYv"

# Configure pyngrok with your authtoken
conf.get_default().auth_token = NGROK_AUTHTOKEN


In [48]:
class ConversationTranscriberWithFixedGuests:
    def __init__(self, subscription_key, region, audio_filename):
        # Initialize Speech Configuration
        self.speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
        self.speech_config.speech_recognition_language = "en-US"

        # Enable speaker diarization (identify different speakers)
        self.speech_config.set_property(
            property_id=speechsdk.PropertyId.SpeechServiceResponse_DiarizeIntermediateResults,
            value='true'
        )

        # Initialize Audio Configuration
        self.audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

        # Initialize Conversation Transcriber
        self.conversation_transcriber = speechsdk.transcription.ConversationTranscriber(
            speech_config=self.speech_config,
            audio_config=self.audio_config
        )

        # Initialize speaker mapping and guest labels
        self.speaker_mapping = {}
        self.guest_labels = ["Guest 1", "Guest 2"]
        self.transcribing_stop = False

        # Store transcription results
        self.transcription_results = []

        # Connect event callbacks
        self.conversation_transcriber.transcribed.connect(self.transcribed_cb)
        self.conversation_transcriber.transcribing.connect(self.transcribing_cb)
        self.conversation_transcriber.session_started.connect(self.session_started_cb)
        self.conversation_transcriber.session_stopped.connect(self.session_stopped_cb)
        self.conversation_transcriber.canceled.connect(self.canceled_cb)
        self.conversation_transcriber.session_stopped.connect(self.stop_cb)
        self.conversation_transcriber.canceled.connect(self.stop_cb)

    def get_guest_label(self, speaker_id):
        """
        Assigns a Guest label to a speaker ID if not already assigned.
        Limits to Guest 1, Guest 2, and dynamically assigns new guests.
        """
        if not speaker_id:
            # Treat empty or None speaker_id as None
            speaker_id = "None"

        if speaker_id not in self.speaker_mapping:
            if len(self.speaker_mapping) < 2:
                # Assign next available guest label
                self.speaker_mapping[speaker_id] = self.guest_labels[len(self.speaker_mapping)]
            else:
                # Assign 'Unknown' for unidentified speakers
                self.speaker_mapping[speaker_id] = "Unknown" if speaker_id == "None" else f"Guest {len(self.speaker_mapping) + 1}"

        return self.speaker_mapping[speaker_id]

    def transcribed_cb(self, evt: speechsdk.SpeechRecognitionEventArgs):
        """
        Callback for when a segment of speech has been transcribed.
        """
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_label = self.get_guest_label(evt.result.speaker_id)
            result_text = f"{speaker_label}: {evt.result.text}"
            st.write(result_text)  # Display in Streamlit
            self.transcription_results.append(result_text)
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            # Optionally handle NoMatch cases if needed
            pass  # Ignoring NoMatch for cleaner output

    def transcribing_cb(self, evt: speechsdk.SpeechRecognitionEventArgs):
        """
        Callback for when the transcription is ongoing (intermediate results).
        """
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_label = self.get_guest_label(evt.result.speaker_id)
            result_text = f"{speaker_label}: {evt.result.text}"
            st.write(result_text)  # Display in Streamlit
            self.transcription_results.append(result_text)

    def session_started_cb(self, evt: speechsdk.SessionEventArgs):
        """
        Callback for when the transcription session starts.
        """
        st.info("Transcription session started.")

    def session_stopped_cb(self, evt: speechsdk.SessionEventArgs):
        """
        Callback for when the transcription session stops.
        """
        st.info("Transcription session stopped.")

    def canceled_cb(self, evt: speechsdk.SessionEventArgs):
        """
        Callback for when the transcription is canceled.
        """
        st.warning("Transcription session canceled.")

    def stop_cb(self, evt: speechsdk.SessionEventArgs):
        """
        Callback to signal the transcription should stop.
        """
        self.transcribing_stop = True

    def recognize_from_file(self):
        """
        Starts the transcription process and waits for it to complete.
        """
        self.conversation_transcriber.start_transcribing_async()
        st.info("Transcription started...")

        # Wait until the transcription session signals to stop
        while not self.transcribing_stop:
            time.sleep(0.5)

        self.conversation_transcriber.stop_transcribing_async()
        st.info("Transcription stopped.")

        # Save results to PDF and return the PDF bytes
        pdf_bytes = self.save_to_pdf()
        return pdf_bytes

    def save_to_pdf(self):
        """
        Saves the transcription results to a PDF file with different colors for each guest,
        adding a blank line after each guest's transcription.
        """
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=letter)
        elements = []

        # Add title
        styles = getSampleStyleSheet()
        title_style = styles['Title']
        title = Paragraph("Transcription", title_style)
        elements.append(title)
        elements.append(Spacer(1, 12))

        # Define colors for each guest
        guest_colors = {
            "Guest 1": colors.navy,
            "Guest 2": colors.teal,
            "Unknown": colors.red,
        }

        # Define a style for the paragraphs
        para_style = styles['BodyText']
        para_style.alignment = 4  # Justified

        for line in self.transcription_results:
            # Get the guest label from the line
            guest_label = line.split(":")[0]  # Assuming format "Guest X: text"

            # Set the text color based on the guest label
            para_style.textColor = guest_colors.get(guest_label, colors.black)

            # Create a paragraph with the current line
            paragraph = Paragraph(line, para_style)
            elements.append(paragraph)
            elements.append(Spacer(1, 12))  # Add a blank line after each guest's transcription

        # Build the PDF
        doc.build(elements)
        buffer.seek(0)
        st.success("PDF generated successfully.")
        return buffer


In [49]:
import base64

def main():
    st.title("AudioLens")

    st.sidebar.header("Azure Credentials")
    subscription_key = st.sidebar.text_input("Azure Subscription Key", type="password")
    region = st.sidebar.text_input("Azure Region", value="centralindia")

    st.header("Upload Audio File")
    uploaded_file = st.file_uploader("Choose a WAV audio file", type=["wav"])

    if st.button("Start Transcription"):
        if not subscription_key:
            st.error("Please enter your Azure Subscription Key.")
            return
        if not region:
            st.error("Please enter your Azure Region.")
            return
        if not uploaded_file:
            st.error("Please upload an audio file.")
            return

        # Save the uploaded file to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_file:
            tmp_file.write(uploaded_file.read())
            audio_filename = tmp_file.name

        try:
            # Initialize the transcriber with your credentials and audio file
            transcriber = ConversationTranscriberWithFixedGuests(subscription_key, region, audio_filename)

            # Start the transcription process
            pdf_buffer = transcriber.recognize_from_file()

            # Provide a download link for the PDF
            st.download_button(
                label="Download Transcription PDF",
                data=pdf_buffer,
                file_name="TranscriptionOutput.pdf",
                mime="application/pdf"
            )

        except speechsdk.CognitiveServicesSpeechException as e:
            st.error(f"Speech SDK Error: {e}")
        except Exception as err:
            st.error(f"Encountered exception: {err}")
        finally:
            # Clean up the temporary file
            os.remove(audio_filename)

if __name__ == "__main__":
    main()


2024-10-16 11:10:27.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.415 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.421 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.425 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 11:10:27.427 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [50]:
# Define your ngrok authtoken
NGROK_AUTHTOKEN = "2nVqzn0Zd0IRxRssPVWIeWd9I7m_3JVCAYER33hYbbXni2fYv"  # Replace with your actual ngrok authtoken

# Combine all parts into the app.py file
app_code = f"""
import os
import time
import streamlit as st
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import azure.cognitiveservices.speech as speechsdk
from pyngrok import ngrok, conf
import tempfile
import io

# Configure pyngrok with your authtoken
conf.get_default().auth_token = "{NGROK_AUTHTOKEN}"

class ConversationTranscriberWithFixedGuests:
    def __init__(self, subscription_key, region, audio_filename):
        # Initialize Speech Configuration
        self.speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
        self.speech_config.speech_recognition_language = "en-US"

        # Enable speaker diarization (identify different speakers)
        self.speech_config.set_property(
            property_id=speechsdk.PropertyId.SpeechServiceResponse_DiarizeIntermediateResults,
            value='true'
        )

        # Initialize Audio Configuration
        self.audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

        # Initialize Conversation Transcriber
        self.conversation_transcriber = speechsdk.transcription.ConversationTranscriber(
            speech_config=self.speech_config,
            audio_config=self.audio_config
        )

        # Initialize speaker mapping and guest labels
        self.speaker_mapping = {{}}
        self.guest_labels = ["Guest 1", "Guest 2"]
        self.transcribing_stop = False

        # Store transcription results
        self.transcription_results = []

        # Connect event callbacks
        self.conversation_transcriber.transcribed.connect(self.transcribed_cb)
        self.conversation_transcriber.transcribing.connect(self.transcribing_cb)
        self.conversation_transcriber.session_started.connect(self.session_started_cb)
        self.conversation_transcriber.session_stopped.connect(self.session_stopped_cb)
        self.conversation_transcriber.canceled.connect(self.canceled_cb)
        self.conversation_transcriber.session_stopped.connect(self.stop_cb)
        self.conversation_transcriber.canceled.connect(self.stop_cb)

    def get_guest_label(self, speaker_id):
        \"""
        Assigns a Guest label to a speaker ID if not already assigned.
        Limits to Guest 1, Guest 2, and dynamically assigns new guests.
        \"""
        if not speaker_id:
            # Treat empty or None speaker_id as None
            speaker_id = "None"

        if speaker_id not in self.speaker_mapping:
            if len(self.speaker_mapping) < 2:
                # Assign next available guest label
                self.speaker_mapping[speaker_id] = self.guest_labels[len(self.speaker_mapping)]
            else:
                # Assign 'Unknown' for unidentified speakers
                self.speaker_mapping[speaker_id] = "Unknown" if speaker_id == "None" else f"Guest {{len(self.speaker_mapping) + 1}}"

        return self.speaker_mapping[speaker_id]

    def transcribed_cb(self, evt: speechsdk.SpeechRecognitionEventArgs):
        \"""
        Callback for when a segment of speech has been transcribed.
        \"""
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_label = self.get_guest_label(evt.result.speaker_id)
            result_text = f"{{speaker_label}}: {{evt.result.text}}"
            st.write(result_text)  # Display in Streamlit
            self.transcription_results.append(result_text)
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            # Optionally handle NoMatch cases if needed
            pass  # Ignoring NoMatch for cleaner output

    def transcribing_cb(self, evt: speechsdk.SpeechRecognitionEventArgs):
        \"""
        Callback for when the transcription is ongoing (intermediate results).
        \"""
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            speaker_label = self.get_guest_label(evt.result.speaker_id)
            result_text = f"{{speaker_label}}: {{evt.result.text}}"
            st.write(result_text)  # Display in Streamlit
            self.transcription_results.append(result_text)

    def session_started_cb(self, evt: speechsdk.SessionEventArgs):
        \"""
        Callback for when the transcription session starts.
        \"""
        st.info("Transcription session started.")

    def session_stopped_cb(self, evt: speechsdk.SessionEventArgs):
        \"""
        Callback for when the transcription session stops.
        \"""
        st.info("Transcription session stopped.")

    def canceled_cb(self, evt: speechsdk.SessionEventArgs):
        \"""
        Callback for when the transcription is canceled.
        \"""
        st.warning("Transcription session canceled.")

    def stop_cb(self, evt: speechsdk.SessionEventArgs):
        \"""
        Callback to signal the transcription should stop.
        \"""
        self.transcribing_stop = True

    def recognize_from_file(self):
        \"""
        Starts the transcription process and waits for it to complete.
        \"""
        self.conversation_transcriber.start_transcribing_async()
        st.info("Transcription started...")

        # Wait until the transcription session signals to stop
        while not self.transcribing_stop:
            time.sleep(0.5)

        self.conversation_transcriber.stop_transcribing_async()
        st.info("Transcription stopped.")

        # Save results to PDF and return the PDF bytes
        pdf_bytes = self.save_to_pdf()
        return pdf_bytes

    def save_to_pdf(self):
        \"""
        Saves the transcription results to a PDF file with different colors for each guest,
        adding a blank line after each guest's transcription.
        \"""
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=letter)
        elements = []

        # Add title
        styles = getSampleStyleSheet()
        title_style = styles['Title']
        title = Paragraph("Transcription", title_style)
        elements.append(title)
        elements.append(Spacer(1, 12))

        # Define colors for each guest
        guest_colors = {{
            "Guest 1": colors.navy,
            "Guest 2": colors.teal,
            "Unknown": colors.red,
        }}

        # Define a style for the paragraphs
        para_style = styles['BodyText']
        para_style.alignment = 4  # Justified

        for line in self.transcription_results:
            # Get the guest label from the line
            guest_label = line.split(":")[0]  # Assuming format "Guest X: text"

            # Set the text color based on the guest label
            para_style.textColor = guest_colors.get(guest_label, colors.black)

            # Create a paragraph with the current line
            paragraph = Paragraph(line, para_style)
            elements.append(paragraph)
            elements.append(Spacer(1, 12))  # Add a blank line after each guest's transcription

        # Build the PDF
        doc.build(elements)
        buffer.seek(0)
        st.success("PDF generated successfully.")
        return buffer

def main():
    st.title("AudioLens")

    st.sidebar.header("Azure Credentials")
    subscription_key = st.sidebar.text_input("Azure Subscription Key", type="password")
    region = st.sidebar.text_input("Azure Region", value="centralindia")

    st.header("Upload Audio File")
    uploaded_file = st.file_uploader("Choose a WAV audio file", type=["wav"])

    if st.button("Start Transcription"):
        if not subscription_key:
            st.error("Please enter your Azure Subscription Key.")
            return
        if not region:
            st.error("Please enter your Azure Region.")
            return
        if not uploaded_file:
            st.error("Please upload an audio file.")
            return

        # Save the uploaded file to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_file:
            tmp_file.write(uploaded_file.read())
            audio_filename = tmp_file.name

        try:
            # Initialize the transcriber with your credentials and audio file
            transcriber = ConversationTranscriberWithFixedGuests(subscription_key, region, audio_filename)

            # Start the transcription process
            pdf_buffer = transcriber.recognize_from_file()

            # Provide a download link for the PDF
            st.download_button(
                label="Download Transcription PDF",
                data=pdf_buffer,
                file_name="TranscriptionOutput.pdf",
                mime="application/pdf"
            )

        except speechsdk.CognitiveServicesSpeechException as e:
            st.error(f"Speech SDK Error: {{e}}")
        except Exception as err:
            st.error(f"Encountered exception: {{err}}")
        finally:
            # Clean up the temporary file
            os.remove(audio_filename)

if __name__ == "__main__":
    main()
"""

# Save the Streamlit app code to a Python file
with open("app.py", "w") as f:
    f.write(app_code)


In [51]:
from pyngrok import ngrok

# Set your new authtoken here
ngrok.set_auth_token("2nVqzn0Zd0IRxRssPVWIeWd9I7m_3JVCAYER33hYbbXni2fYv")


In [52]:
from pyngrok import ngrok

# Create the tunnel and get the public URL
public_url = ngrok.connect(8501)
print(f"Streamlit app is available at: {public_url}")


Streamlit app is available at: NgrokTunnel: "https://dc24-34-74-10-1.ngrok-free.app" -> "http://localhost:8501"
